In [1]:
import os

In [3]:
!pip install langchain-groq


  Using cached langchain_groq-0.1.10-py3-none-any.whl.metadata (2.9 kB)
Using cached langchain_groq-0.1.10-py3-none-any.whl (14 kB)


In [3]:

import groq
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks.base import BaseCallbackHandler
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2


In [4]:
load_dotenv()

True

In [6]:
key=os.getenv("GROQ_API_KEY")

In [7]:
llm = ChatGroq(groq_api_key=key,model_name="llama3-8b-8192",temperature=0.7  # Directly pass the API key
)

In [11]:
with open("Response.json","r") as f:
    RESPONSE_JSON=json.load(f)

In [9]:
TEMPLATE="""
Text:{text}
 You are an expert MCQ maker. Given the above text, it is your job to \
    create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone.
    Make sure the questions are not repeated and check all the questions to be conforming the text as well.
    Make sure to format your response like  RESPONSE_JSON below  and use it as a guide.
    Ensure to make {number} MCQs
    ### RESPONSE_JSON
    {RESPONSE_JSON}

    """

In [16]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    template=TEMPLATE
)

In [17]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_1008\3899119271.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)


In [ ]:
TEMPLATE2="""
 You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
 You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.
 if the quiz is not at per with the cognitive and analytical abilities of the students,\
 update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
 Quiz_MCQs:
 {quiz}

 Check from an expert English Writer of the above quiz
 """